`Dataframe` to `DocumentArray`...

In [ ]:
from docarray import Document, DocumentArray
import pandas

from k_fold import data_selection

train_df, test_df = data_selection('/Users/hanselblanco/Documents/4to/ML/project/bias-project-ML/datasets/utkface')

train_da = DocumentArray.from_dataframe(train_df)
test_da = DocumentArray.from_dataframe(test_df)

# for item in train_da:
#     item.summary()


Initializing `CLIP`...

In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

`CLIP` Fine-tuning...

In [ ]:
import finetuner

finetuner.login()

`Race` Fine-tuning...

In [ ]:
race_pairs = DocumentArray() # initialize a DocumentArray as final training data.

prompt = 'This is a person of race '
for doc in train_da:
    pair = Document()
    doc.uri = doc.tags['filepath']
    img_chunk = doc.load_uri_to_image_tensor(224, 224)
    img_chunk.modality = 'image'
    txt_chunk = Document(content=prompt + doc.tags['race'])
    txt_chunk.modality = 'text'
    pair.chunks.extend([img_chunk, txt_chunk])
    # add pair to pairs
    race_pairs.append(pair)

# Lets see the first item of the pairs
race_pairs[0]

In [ ]:
len(race_pairs)

`Gender` Fine-tuning...

In [ ]:
gender_pairs = DocumentArray() # initialize a DocumentArray as final training data.

prompt = 'This is a person of gender '
for doc in train_da:
    pair = Document()
    doc.uri = doc.tags['filepath']
    img_chunk = doc.load_uri_to_image_tensor(224, 224)
    img_chunk.modality = 'image'
    txt_chunk = Document(content=prompt + doc.tags['gender'])
    txt_chunk.modality = 'text'
    pair.chunks.extend([img_chunk, txt_chunk])
    # add pair to pairs
    gender_pairs.append(pair)

# Lets see the first item of the pairs
gender_pairs[0]

In [ ]:
len(gender_pairs)

Training `CLIP` model...

In [ ]:
run = finetuner.fit(
    model='openai/clip-vit-base-patch32', # fine-tune CLIP
    train_data=race_pairs,   
    learning_rate=1e-5,
    loss='CLIPLoss',
    cpu=False,
)

In [ ]:
for entry in run.stream_logs():
    print(entry)

Saving the artifact

In [ ]:
artifact = run.save_artifact('/content/')

In [ ]:
clip_text_encoder = finetuner.get_model(artifact=artifact, select_model='clip-text')
clip_image_encoder = finetuner.get_model(artifact=artifact, select_model='clip-vision')